In [14]:
#Retail & E-Commerce

# Instacart Market Basket Analysis
# Kaggle

#data
#https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis/data

#class data doc
#https://docs.google.com/document/d/1BsJIWU4TKUkADvnNEC14b8coQ5VqslJtS7ZBCYJVZAk/edit?tab=t.ta49d25hofw5#heading=h.bbadcysas44j

# Week 2 - Preprocessing, part 2

# 1. Lesson: None

# 2. Weekly graph question

The Storytelling With Data book mentions planning on a "Who, What, and How" for your data story.  Write down a possible Who, What, and How for your data, using the ideas in the book.

In [15]:
'''
who: business leaders wanting to sell products before expire or easons?
what: we want to sell more of x product
how: ill show what products or areas could be better
'''

'\nwho: business leaders wanting to sell products before expire or easons?\nwhat: we want to sell more of x product\nhow: ill show what products or areas could be better\n'

# 3. Homework - work with your own data

This week, you will do the same types of exercises as last week, but you should use your chosen datasets that someone in your class found last semester. (They likely will not be the particular datasets that you found yourself.)

### Here are some types of analysis you can do  Use Google, documentation, and ChatGPT to help you:

- Summarize the datasets using info() and describe()

- Are there any duplicate rows?

- Are there any duplicate values in a given column (when this would be inappropriate?)

- What are the mean, median, and mode of each column?

- Are there any missing or null values?

    - Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?

- Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)

- Encode any categorical variables (e.g. with one-hot encoding.)

### Conclusions:

- Are the data usable?  If not, find some new data!

- Do you need to modify or correct the data in some way?

- Is there any class imbalance?  (Categories that have many more items than other categories).

<hr>


#### Load Data & Libs

In [16]:
#!pip install pandas
#!pip install pyarrow


In [17]:
import pandas as pd
import numpy as np
import pyarrow as pa
from datetime import datetime, timedelta

import glob
import os

In [18]:


base = "../data/Retail_and_ECommerce/Instacart_Market_Basket_A"

# # ---- Simple files ----
# aisles = pd.read_csv(os.path.join(base, "aisles.csv"))
# departments = pd.read_csv(os.path.join(base, "departments.csv"))
# products = pd.read_csv(os.path.join(base, "products.csv"))
# order_products_train = pd.read_csv(os.path.join(base, "order_products__train.csv"))

# # ---- Combine split files ----

# # orders parts
# orders_files = glob.glob(os.path.join(base, "orders_part_*.csv"))
# orders = pd.concat([pd.read_csv(f) for f in orders_files], ignore_index=True)

# # prior parts
# prior_files = glob.glob(os.path.join(base, "order_products__prior_*.csv"))
# order_products_prior = pd.concat([pd.read_csv(f) for f in prior_files], ignore_index=True)

# print("Loaded everything.")
# print("orders shape:", orders.shape)
# print("order_products_prior shape:", order_products_prior.shape)


##### Merge the data

In [19]:
# # Combine prior and train if you want full dataset
# order_products = pd.concat(
#     [order_products_prior, order_products_train],
#     axis=0
# )

# # Merge with orders
# df = order_products.merge(orders, on="order_id", how="left")

# # Merge with products
# df = df.merge(products, on="product_id", how="left")

# # Merge with aisles
# df = df.merge(aisles, on="aisle_id", how="left")

# # Merge with departments
# df = df.merge(departments, on="department_id", how="left")

# df.head()

#### Save merged dataset

In [20]:
#out_path = os.path.join(base, "instacart_merged.parquet")
#df.to_parquet(out_path, index=False)

In [21]:
out_path = os.path.join(base, "instacart_merged.parquet")
df = pd.read_parquet(out_path)


#### Begin EDA

In [29]:
#Summarize the datasets using info() and describe()

df.info()

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   product_id              int64  
 2   add_to_cart_order       int64  
 3   reordered               int64  
 4   user_id                 int64  
 5   eval_set                object 
 6   order_number            int64  
 7   order_dow               int64  
 8   order_hour_of_day       int64  
 9   days_since_prior_order  float64
 10  product_name            object 
 11  aisle_id                int64  
 12  department_id           int64  
 13  aisle                   object 
 14  department              object 
dtypes: float64(1), int64(10), object(4)
memory usage: 3.8+ GB


,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
count,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07,3.174104e+07,3.381911e+07,3.381911e+07
mean,1.710566e+06,2.557551e+04,8.367738e+00,5.900617e-01,1.029444e+05,1.713998e+01,2.737285e+00,1.343123e+01,1.136415e+01,7.121799e+01,9.918544e+00
std,9.874008e+05,1.409770e+04,7.139540e+00,4.918220e-01,5.946733e+04,1.749829e+01,2.093296e+00,4.246149e+00,8.940500e+00,3.819898e+01,6.281655e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,8.554130e+05,1.351900e+04,3.000000e+00,0.000000e+00,5.143500e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,3.100000e+01,4.000000e+00
50%,1.710660e+06,2.525600e+04,6.000000e+00,1.000000e+00,1.026260e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,8.300000e+01,9.000000e+00
75%,2.565587e+06,3.793500e+04,1.100000e+01,1.000000e+00,1.544120e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,1.070000e+02,1.600000e+01
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01,1.340000e+02,2.100000e+01


In [23]:
#Are there any duplicate rows?



In [24]:
#Are there any duplicate values in a given column (when this would be inappropriate?)



In [25]:
#What are the mean, median, and mode of each column?



In [26]:
#Are there any missing or null values?
####Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?



In [27]:
#Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)


In [28]:
#Encode any categorical variables (e.g. with one-hot encoding.)

# 4. Storytelling With Data graph

Just like last week: choose any graph in the Introduction of Storytelling With Data. Use matplotlib to reproduce it in a rough way. I don't expect you to spend an enormous amount of time on this; I understand that you likely will not have time to re-create every feature of the graph. However, if you're excited about learning to use matplotlib, this is a good way to do that. You don't have to duplicate the exact values on the graph; just the same rough shape will be enough.  If you don't feel comfortable using matplotlib yet, do the best you can and write down what you tried or what Google searches you did to find the answers.